## LangChain Expression Language 中最重要的接口 Runnable
LEL ( LangChain Expression Language ) 是一种以声明式方法，轻松地将链或组件组合在一起的机制。今天我们来介绍 LEL 中最重要的接口 Runnable。

## Runnable
LangChain 定义了 Runnable 接口，绝大多数组件也实现了该接口。Runnable 接口定义了如下函数：

- stream: 流式输出响应
- invoke: 基于单一输入调用链
- batch: 基于一组输入调用链
- astream
- ainvoke
- abatch
后三个函数为前三个函数的异步版本。更多内容请参考 Expression Language Interface。

ChatPromptTemplate 与 ChatOpenAI 类都实现了 Runnable 接口。参考如下代码：

In [2]:
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import Runnable

prompt = ChatPromptTemplate.from_template("Hi, LEL!")
print(isinstance(prompt, Runnable))

True


## RunnablePassthrough 在管道中实现输入的传递
在通过管道构建LangChain链时，我们可能需要将原始输入变量传递给链式模型的后续步骤。我们可以使用类 RunnablePassthrough 来达到输入传递的目的。请参考一下示例：

RunnablePassthrough 接受输入，并如实地将输入作为自己的输出，从而达到传递的目的。这里我们实现一个 Runnable 来接受一个 Dict 类型的输入，并在控制台打印出键为 name 的值，以此来测试 RunnablePassthrough 的传递效果。

在构成的链中，第一个 RunnablePassthrough 传递的是 chain.invoke("Alex") 中的字符串参数 Alex，第二个 RunnablePassthrough 传递的是管道第一部分的输出，一个字典 dict:

{
  "name": "Alex"
}

In [4]:
from langchain.schema.runnable import RunnablePassthrough, RunnableConfig
from langchain.load.serializable import Serializable
from typing import Optional, Dict
from langchain_core.runnables.utils import Input


# 自定义一个继承Runnable的类
class StdOutputRunnable(Serializable, Runnable[Input, Input]):
    @property
    def lc_serializable(self) -> bool:
        return True

    def invoke(self, input: Dict, config: Optional[RunnableConfig] = None) -> Input:
        # print(f"Hey, I received the name {input['name']}")
        print(input)
        return self._call_with_config(lambda x: x, input, config)


chain = {"name": RunnablePassthrough()} | RunnablePassthrough() | StdOutputRunnable()

# 调的是invoke方法，所以管道里也是看invoke方法
chain.invoke("Simon")


{'name': 'Simon'}


{'name': 'Simon'}

In [20]:
import os
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

load_dotenv(override=True)

DASHSCOPE_API_KEY = os.getenv('DASHSCOPE_API_KEY')

# qwen 兼容 openai的接口
qw_llm_openai = ChatOpenAI(
    openai_api_base='https://dashscope.aliyuncs.com/compatible-mode/v1',
    openai_api_key=DASHSCOPE_API_KEY,
    model_name="qwen2-1.5b-instruct",
    temperature=0.7,
    streaming=True,
)


In [22]:
from langchain_core.runnables.passthrough import (
    RunnableAssign,
    RunnableParallel,
)
from operator import itemgetter
from langchain_core.runnables.base import RunnableLambda
from langchain_core.output_parsers import StrOutputParser


def prompt():
    return '''
    # Character
    You're a knowledgeable assistant capable of providing concise answers to a variety of questions, drawing from the context provided, and admitting when you don't know the answer.
    
    ## Skills
    1. **Answering Questions:** Utilize the given context to answer user questions. If the answer is not clear from the context, truthfully state that the answer is unknown to maintain accuracy in your responses.
    Question: {question}
    Context: {context}    
    2. You are a nice chatbot having a conversation with a human.
    Previous conversation:
    {chat_history}
    
    ### Answering Questions Format:
    - Answer:  
    
    ## Constraints:
    - Keep answers to a maximum of three sentences to maintain brevity.
    - If the answer cannot be determined, simply confess that you do not know. Honesty is paramount in maintaining credibility.
    - If the answer is not reflected in the context, please reply: Sorry, I don't know for the moment.
    - Focus on gleaning answers from the context provided only.
    - All questions should be answered in Chinese
    '''


def fake_llm(prompt: str) -> Dict:
    return {
        "context": 'context context context',
        "question": prompt,
        "chat_history": 'historyhistoryhistoryhistoryhistory'
    }


chain = (
        RunnableLambda(fake_llm) |
        {
            "context": itemgetter('context'),
            "question": itemgetter('question'),
            "chat_history": itemgetter('chat_history')
        }
        | ChatPromptTemplate.from_template(prompt())
        | RunnablePassthrough()
        | StdOutputRunnable()
        | qw_llm_openai
        | StrOutputParser()
)

chain.invoke('Simon')

messages=[HumanMessage(content="\n    # Character\n    You're a knowledgeable assistant capable of providing concise answers to a variety of questions, drawing from the context provided, and admitting when you don't know the answer.\n    \n    ## Skills\n    1. **Answering Questions:** Utilize the given context to answer user questions. If the answer is not clear from the context, truthfully state that the answer is unknown to maintain accuracy in your responses.\n    Question: Simon\n    Context: context context context    \n    2. You are a nice chatbot having a conversation with a human.\n    Previous conversation:\n    historyhistoryhistoryhistoryhistory\n    \n    ### Answering Questions Format:\n    - Answer:  \n    \n    ## Constraints:\n    - Keep answers to a maximum of three sentences to maintain brevity.\n    - If the answer cannot be determined, simply confess that you do not know. Honesty is paramount in maintaining credibility.\n    - If the answer is not reflected in the

'Simon是您要找的人吗？请问您需要关于Simon的什么信息呢？'

## itemgetter 实现输入的部分传递
我们可以不需要传递输入的完整数据。当我们想要传递字典中的某一个键值，可以通过 itemgetter 函数实现。请参考如下代码：

In [13]:
def fake_llm(prompt: str) -> Dict:  # Fake LLM for the example
    return {'aa': '1123', 'vv': '63'}


RunnableLambda(fake_llm).invoke('ee')

{'aa': '1123', 'vv': '63'}

In [10]:
runnable = RunnableParallel(
    origin=RunnablePassthrough(),
    modified=lambda x: x + 1
)

runnable.invoke(1)

TypeError: Expected a Runnable, callable or dict.Instead got an unsupported type: <class 'str'>

In [10]:
from langchain.schema import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough, RunnableConfig
from langchain_core.runnables.utils import Input
from langchain.load.serializable import Serializable
from operator import itemgetter

chain = {"name": itemgetter("user_name")} | RunnablePassthrough() | StdOutputRunnable()

# user_name 就是传递的key
chain.invoke({"user_name": "Alex"})


NameError: name 'StdOutputRunnable' is not defined

## 连接模型的一个完整例子
1. 利用 Retriever 增加外部数据获取能力
2. 管道连接 OpenAI 的聊天模型完成问答

In [1]:

from langchain.vectorstores import Chroma
from dotenv import load_dotenv
import os

from langchain_community.embeddings.cloudflare_workersai import (
    CloudflareWorkersAIEmbeddings,
)

load_dotenv(override=True)
account_id = os.getenv('CF_ACCOUNT_ID')
api_token = os.getenv('CF_API_TOKEN')
# @cf/baai/bge-large-en-v1.5
# 维度是：1024

# @cf/baai/bge-small-en-v1.5
# 维度是：384
embeddings = CloudflareWorkersAIEmbeddings(
    account_id=account_id,
    api_token=api_token,
    model_name="@cf/baai/bge-small-en-v1.5",
)

In [2]:
vectorstore = Chroma.from_texts(["My name is VerySmallWoods, a software engineer based in Dublin."],
                                embedding=embeddings)

In [3]:
retriever = vectorstore.as_retriever()

In [4]:
retriever.__class__

langchain_core.vectorstores.VectorStoreRetriever

langchain.vectorstores.base.VectorStoreRetriever
VectorStoreRetriever 的 BaseRetriever 基类实现了 invoke 函数。它接受字符串类型输入，并调用 get_relevant_documents 函数查询相关文档。

class BaseRetriever(Serializable, Runnable[str, List[Document]], ABC):
    # ......
    def invoke(
        self, input: str, config: Optional[RunnableConfig] = None
    ) -> List[Document]:
        return self.get_relevant_documents(input, **(config or {}))

In [11]:
from langchain_community.llms.cloudflare_workersai import CloudflareWorkersAI

model = '@cf/meta/llama-3-8b-instruct'
cf_llm = CloudflareWorkersAI(account_id=account_id, api_token=api_token, model=model)

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

chain = ({
             "context": retriever,
             "question": RunnablePassthrough()
         } | prompt | cf_llm | StrOutputParser())

In [12]:
chain.invoke("Who am I?")

Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1


'Based on the context provided, your name is VerySmallWoods, and you are a software engineer based in Dublin.'

In [13]:
chain.invoke("Where do I live?")

Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1


ConnectionError: HTTPSConnectionPool(host='api.cloudflare.com', port=443): Max retries exceeded with url: /client/v4/accounts/8483c3ec7a0cbc54a8d660b5b9002b04/ai/run/@cf/meta/llama-3-8b-instruct (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x109cf2a40>: Failed to resolve 'api.cloudflare.com' ([Errno 8] nodename nor servname provided, or not known)"))